In [1]:
import pandas as pd
import numpy as np

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Reshape, LayerNormalization, MultiHeadAttention, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

In [3]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,  f1_score, accuracy_score

In [4]:
selected_features = ['destination_port',
            'init_win_bytes_backward',
            'init_win_bytes_forward',
            'bwd_packets_s',
            'fwd_iat_min',
            'min_seg_size_forward',
            'flow_iat_min',
            'flow_duration',
            'total_length_of_fwd_packets',
            'total_backward_packets',
            'bwd_iat_min',
            'bwd_packet_length_std',
            'fwd_iat_total',
            'fwd_packet_length_mean',
            'fwd_packet_length_max',
            'flow_iat_std',
            'fwd_packets_s',
            'down_up_ratio',
            'total_fwd_packets',
            'bwd_packet_length_min',
            'flow_bytes_s',
            'bwd_header_length',
            'packet_length_mean',
            'total_length_of_bwd_packets']

In [5]:
def change_datatype(df):
    int8_vals = np.iinfo(np.int8)
    int16_vals = np.iinfo(np.int16)
    int32_vals = np.iinfo(np.int32)

    float16_vals = np.finfo(np.float16)
    float32_vals = np.finfo(np.float32)
    
    for col in df.columns:
        max_val = df[col].max()
        min_val = df[col].min()

        # print(f"{col}: max {max_val} -- min {min_val}")
        
        if df[col].dtype == np.int64:
            if max_val <= int8_vals.max and min_val >= int8_vals.min:
                df[col] = df[col].astype(np.int8)
            elif max_val <= int16_vals.max and min_val >= int16_vals.min:
                df[col] = df[col].astype(np.int16)
            elif max_val <= int32_vals.max and min_val >= int32_vals.min:
                df[col] = df[col].astype(np.int32)

        elif df[col].dtype == np.float64:
            if max_val <= float16_vals.max and min_val >= float16_vals.min:
                df[col] = df[col].astype(np.float16)
            elif max_val <= float32_vals.max and min_val >= float32_vals.min:
                df[col] = df[col].astype(np.float32)
    return df

In [6]:
df_train = change_datatype(pd.read_csv("./oversampling/NRAS_trainset.csv"))
df_test = change_datatype(pd.read_csv("./raw/raw_testset.csv"))

df_train = df_train[selected_features + ['label']]
df_test = df_test[selected_features + ['label']]

In [7]:
df_train.shape

(9539966, 25)

In [8]:
df_test.shape

(756690, 25)

In [9]:
scaler = QuantileTransformer(
       n_quantiles = 10000,
       random_state = 6969,
       output_distribution = "uniform"
)

In [10]:
X_train = df_train.drop('label', axis = 1)
y_train = df_train['label']

X_test = df_test.drop('label', axis = 1)
y_test = df_test['label']

In [11]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train.shape

In [12]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test.shape

(756690, 24, 1)

## CNN Model

In [13]:
model = Sequential([
    Input(shape = (X_train.shape[1], 1)),
    Conv1D(512, 3, padding = "same", activation = "relu"),
    LayerNormalization(),
    
    MaxPooling1D(pool_size = (2)),
    Dropout(0.5),
    
    Conv1D(128, 3, padding = "same", activation = "relu"),
    MaxPooling1D(pool_size = (2)),
    Dropout(0.5),
    
    Conv1D(64, 4, padding = "same", activation = "relu"),
    MaxPooling1D(pool_size = (2)),
    Dropout(0.5),
    
    Flatten(),
    Dense(128, activation = "relu"),
    Dropout(0.3),
    
    Dense(y_train.unique().shape[0], activation="softmax")
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 24, 512)           2048      
                                                                 
 layer_normalization (Layer  (None, 24, 512)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 12, 512)           0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 512)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 12, 128)           196736    
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 6, 128)            0

In [15]:
model.compile(
        loss = SparseCategoricalCrossentropy(),
        optimizer = Adam(learning_rate = 0.001),
        metrics = ['accuracy']
    )

In [19]:
checkpoint = ModelCheckpoint("./checkpoint/model_weights_oversampling.hdf5", 
                             monitor = 'val_loss', verbose = 1, 
                             save_best_only = True, mode = 'min')

In [17]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 5, 
                              min_lr = 1e-5, cooldown = 1, mode = "min", verbose = 1)

In [20]:
hist = model.fit(X_train, y_train,
                    validation_data = (X_test, y_test), 
                    epochs = 15, batch_size = 512, 
                    callbacks = [checkpoint, reduce_lr])

Epoch 1/100
18633/18633 [==============================] - ETA: 0s - loss: 0.1035 - accuracy: 0.9554
Epoch 1: val_loss improved from inf to 0.05108, saving model to ./checkpoint\model_weights_oversampling.hdf5
18633/18633 [==============================] - 8894s 477ms/step - loss: 0.1035 - accuracy: 0.9554 - val_loss: 0.0511 - val_accuracy: 0.9791 - lr: 0.0010


d:\dataset\env\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/100
18633/18633 [==============================] - ETA: 0s - loss: 0.0753 - accuracy: 0.9659
Epoch 2: val_loss improved from 0.05108 to 0.04729, saving model to ./checkpoint\model_weights_oversampling.hdf5
18633/18633 [==============================] - 8505s 456ms/step - loss: 0.0753 - accuracy: 0.9659 - val_loss: 0.0473 - val_accuracy: 0.9841 - lr: 0.0010
Epoch 3/100
18633/18633 [==============================] - ETA: 0s - loss: 0.0645 - accuracy: 0.9709
Epoch 3: val_loss improved from 0.04729 to 0.03976, saving model to ./checkpoint\model_weights_oversampling.hdf5
18633/18633 [==============================] - 8159s 438ms/step - loss: 0.0645 - accuracy: 0.9709 - val_loss: 0.0398 - val_accuracy: 0.9874 - lr: 0.0010
Epoch 4/100
18633/18633 [==============================] - ETA: 0s - loss: 0.0570 - accuracy: 0.9738
Epoch 4: val_loss improved from 0.03976 to 0.03863, saving model to ./checkpoint\model_weights_oversampling.hdf5
18633/18633 [==============================] - 10916